## 테스트 데이터셋 로드

In [1]:
import os
import urllib.request
import tarfile
import pickle
import numpy as np
from common.MiniVGGNet import MiniVGGNet

def download_cifar100(save_path='cifar-100-python'):
    if os.path.exists(save_path):
        print("CIFAR-100 이미 존재")
        return

    url = 'https://www.cs.toronto.edu/~kriz/cifar-100-python.tar.gz'
    filename = 'cifar-100-python.tar.gz'
    print("CIFAR-100 다운로드 중...")
    urllib.request.urlretrieve(url, filename)

    with tarfile.open(filename, 'r:gz') as tar:
        tar.extractall()
    os.remove(filename)
    print("다운로드 완료")

def load_batch(filepath):
    with open(filepath, 'rb') as f:
        data_dict = pickle.load(f, encoding='bytes')
    data = data_dict[b'data']
    fine_labels = np.array(data_dict[b'fine_labels'])
    coarse_labels = np.array(data_dict[b'coarse_labels'])
    data = data.reshape(-1, 3, 32, 32).astype(np.float32) / 255.0

    mean = np.array([0.5071, 0.4865, 0.4409]).reshape(1, 3, 1, 1)
    std = np.array([0.2673, 0.2564, 0.2762]).reshape(1, 3, 1, 1)
    data = (data - mean) / std

    return data, fine_labels, coarse_labels

In [2]:
def load_cifar100_full(data_dir='./cifar-100-python', valid_ratio=0.1):
    # 1. Load train and test batch
    x_train_all, y_train_fine_all, y_train_coarse_all = load_batch(os.path.join(data_dir, 'train'))
    x_test, y_test_fine, y_test_coarse = load_batch(os.path.join(data_dir, 'test'))

    # 2. Split train → train + validation
    num_total = x_train_all.shape[0]
    num_valid = int(num_total * valid_ratio)

    x_valid = x_train_all[:num_valid]
    t_valid_fine = y_train_fine_all[:num_valid]
    t_valid_coarse = y_train_coarse_all[:num_valid]

    x_train = x_train_all[num_valid:]
    t_train_fine = y_train_fine_all[num_valid:]
    t_train_coarse = y_train_coarse_all[num_valid:]

    # 3. Return all
    return (x_train, x_valid, x_test,
            t_train_coarse, t_valid_coarse, y_test_coarse,
            t_train_fine, t_valid_fine, y_test_fine)

In [3]:
# CIFAR-100 다운로드
download_cifar100()

# 전체 데이터셋 로드
(x_train, x_valid, x_test,
 t_train_coarse, t_valid_coarse, t_test_coarse,
 t_train_fine, t_valid_fine, t_test_fine) = load_cifar100_full()

# 확인 출력
print("CIFAR-100 데이터셋 로드 완료:")
print(" - x_train:", x_train.shape)
print(" - x_valid:", x_valid.shape)
print(" - x_test :", x_test.shape)
print(" - t_train_fine :", t_train_fine.shape)
print(" - t_valid_fine :", t_valid_fine.shape)
print(" - t_test_fine  :", t_test_fine.shape)
print(" - t_train_coarse:", t_train_coarse.shape)
print(" - t_valid_coarse:", t_valid_coarse.shape)
print(" - t_test_coarse :", t_test_coarse.shape)


CIFAR-100 이미 존재
CIFAR-100 데이터셋 로드 완료:
 - x_train: (45000, 3, 32, 32)
 - x_valid: (5000, 3, 32, 32)
 - x_test : (10000, 3, 32, 32)
 - t_train_fine : (45000,)
 - t_valid_fine : (5000,)
 - t_test_fine  : (10000,)
 - t_train_coarse: (45000,)
 - t_valid_coarse: (5000,)
 - t_test_coarse : (10000,)


In [4]:
def restore_model_parameters(model, model_state):
    for name in ['conv1', 'conv2', 'conv3', 'conv4', 'conv5', 'fc1', 'fc2']:
        layer = getattr(model, name)
        if hasattr(layer, 'W'):
            layer.W = model_state[f'{name}_W']
            layer.b = model_state[f'{name}_b']

def restore_bn_params(model, bn_state):
    for idx, name in enumerate(['bn1', 'bn2', 'bn3', 'bn4', 'bn5']):
        bn = getattr(model, name)
        bn.gamma = bn_state[f'bn{idx}_gamma']
        bn.beta = bn_state[f'bn{idx}_beta']
        bn.running_mean = bn_state[f'bn{idx}_running_mean']
        bn.running_var = bn_state[f'bn{idx}_running_var']

In [5]:
model = MiniVGGNet()

with open("MiniVGGNet_final_ex1_best.pkl", "rb") as f:
    checkpoint = pickle.load(f)
    model_state = checkpoint["model"]
    bn_state = checkpoint["bn"]
    
restore_bn_params(model, bn_state)
restore_model_parameters(model, model_state)

In [14]:
print(checkpoint.keys())

dict_keys(['model', 'bn', 'optimizer', 'train_loss_list', 'train_acc_list', 'val_acc_list', 'val_loss_list'])


In [15]:
print("fc2 std:", np.std(model.fc2.W))           # ≈ 0.1~0.3
print("bn1.running_mean[:5]:", model.bn1.running_mean[:5])  # 0이면 복원 실패

fc2 std: 0.020376587951831216
bn1.running_mean[:5]: [  -41.22232356 -1228.74785389  2062.39043984  1563.41479386
 -3176.95858672]


In [16]:
print(np.mean(x_test))  # ≈ 0.0
print(np.std(x_test))   # ≈ 1.0

0.004534291272868822
1.0035214780791422


In [17]:
logits = model.predict(x_test[:10])
print("logits[0] sum:", np.sum(logits[0]))  # softmax 전 결과, sum != 1.0 정상

logits[0] sum: 17.037402995057164


In [7]:
from common.functions import softmax  

def evaluate_model(model, x, y_true):
    batch_size = 100
    preds = []
    for i in range(0, x.shape[0], batch_size):
        x_batch = x[i:i+batch_size]
        logits = model.predict(x_batch)        
        probs = softmax(logits)                
        y_pred = np.argmax(probs, axis=1)
        preds.append(y_pred)
    preds = np.concatenate(preds)
    acc = np.sum(preds == y_true) / len(y_true)
    return preds, acc

#pred : 예측된 클래스 번호
#전체 정확도

In [8]:
fine_preds, fine_acc = evaluate_model(model, x_train, t_train_fine)
print(f"[🎯 Fine Label] Train Accuracy: {fine_acc:.4f}")

[🎯 Fine Label] Train Accuracy: 0.0099


In [9]:
fine_valid_preds, fine_valid_acc = evaluate_model(model, x_valid, t_valid_fine)
print(f"[🎯 Fine Label] Valid Accuracy: {fine_valid_acc:.4f}")

[🎯 Fine Label] Valid Accuracy: 0.0106


In [10]:
fine_test_preds, fine_test_acc = evaluate_model(model, x_test, t_test_fine)
print(f"[🎯 Fine Label] Test Accuracy: {fine_test_acc:.4f}")

[🎯 Fine Label] Test Accuracy: 0.0100


## 매핑 전 필요한거

In [11]:
## train set
import numpy as np
from common.functions import softmax
from common.coarse_mapping import get_cifar100_fine_to_coarse_dict, CoarseMapper, compute_coarse_accuracy, compute_fine_class_accuracy

# 1. fine → coarse 매핑 딕셔너리 및 매퍼 클래스 준비
f2c = get_cifar100_fine_to_coarse_dict()
mapper = CoarseMapper(f2c)

# 2. 모델 출력 얻기
logits = model.predict(x_train)          # (N, 100)
fine_probs = softmax(logits)             # (N, 100)
fine_preds = np.argmax(fine_probs, axis=1)

# 3. 정답 coarse label 변환
true_coarse = np.array([f2c[f] for f in t_train_fine])

# 4. 각 방식별 coarse 예측 및 정확도 계산
# 방식 1: Argmax 기반
pred_coarse_1 = mapper.argmax_mapping(fine_probs)
acc1 = compute_coarse_accuracy(pred_coarse_1, true_coarse)
print(f"[🎯 방식 1 - Argmax Mapping]     Coarse Accuracy: {acc1:.4f}")

# 방식 2: Entropy Weighted Mapping
pred_coarse_2 = mapper.entropy_weighted_mapping(fine_probs)
acc2 = compute_coarse_accuracy(pred_coarse_2, true_coarse)
print(f"[🔍 방식 2 - Entropy Weighted]   Coarse Accuracy: {acc2:.4f}")

# 방식 3: Soft Average Mapping
pred_coarse_3 = mapper.soft_average_mapping(fine_probs)
acc3 = compute_coarse_accuracy(pred_coarse_3, true_coarse)
print(f"[🧪 방식 3 - Soft Average]        Coarse Accuracy: {acc3:.4f}")

# 방식 4: Validation Guided Mapping
# fine class별 accuracy 기반 가중치 구하기
fine_weights = compute_fine_class_accuracy(fine_preds, t_train_fine)
pred_coarse_4 = mapper.validation_guided_mapping(fine_probs, fine_weights)
acc4 = compute_coarse_accuracy(pred_coarse_4, true_coarse)
print(f"[📊 방식 4 - Validation Guided]  Coarse Accuracy: {acc4:.4f}")



[🎯 방식 1 - Argmax Mapping]     Coarse Accuracy: 0.0501
[🔍 방식 2 - Entropy Weighted]   Coarse Accuracy: 0.0501
[🧪 방식 3 - Soft Average]        Coarse Accuracy: 0.0501
[📊 방식 4 - Validation Guided]  Coarse Accuracy: 0.0501


In [12]:
## valid set
import numpy as np
from common.functions import softmax
from common.coarse_mapping import (
    get_cifar100_fine_to_coarse_dict,
    CoarseMapper,
    compute_fine_class_accuracy,
    compute_coarse_accuracy
)

# 1. 딕셔너리 및 coarse 매퍼 초기화
f2c = get_cifar100_fine_to_coarse_dict()
mapper = CoarseMapper(f2c)

# 2. validation set 예측
val_logits = model.predict(x_valid)            # (N, 100)
val_probs = softmax(val_logits)
val_preds = np.argmax(val_probs, axis=1)
true_coarse_val = np.array([f2c[f] for f in t_valid_fine])  # 정답 coarse

# 3. fine accuracy 계산 (자체 validation용 가중치 생성)
fine_weights_val = compute_fine_class_accuracy(val_preds, t_valid_fine)  # (100,)

# 4. coarse mapping 방식별 평가

# [1] Argmax Mapping
pred_coarse_1 = mapper.argmax_mapping(val_probs)
acc1 = compute_coarse_accuracy(pred_coarse_1, true_coarse_val)
print(f"[🎯 방식 1 - Argmax Mapping]     Validation Coarse Accuracy: {acc1:.4f}")

# [2] Entropy Weighted Mapping
pred_coarse_2 = mapper.entropy_weighted_mapping(val_probs)
acc2 = compute_coarse_accuracy(pred_coarse_2, true_coarse_val)
print(f"[🔍 방식 2 - Entropy Weighted]   Validation Coarse Accuracy: {acc2:.4f}")

# [3] Soft Average Mapping
pred_coarse_3 = mapper.soft_average_mapping(val_probs)
acc3 = compute_coarse_accuracy(pred_coarse_3, true_coarse_val)
print(f"[🧪 방식 3 - Soft Average]        Validation Coarse Accuracy: {acc3:.4f}")

# [4] Validation Guided Mapping (val 내부에서 weight 추정)
pred_coarse_4 = mapper.validation_guided_mapping(val_probs, fine_weights_val)
acc4 = compute_coarse_accuracy(pred_coarse_4, true_coarse_val)
print(f"[📊 방식 4 - Validation Guided]  Validation Coarse Accuracy: {acc4:.4f}")


[🎯 방식 1 - Argmax Mapping]     Validation Coarse Accuracy: 0.0488
[🔍 방식 2 - Entropy Weighted]   Validation Coarse Accuracy: 0.0488
[🧪 방식 3 - Soft Average]        Validation Coarse Accuracy: 0.0488
[📊 방식 4 - Validation Guided]  Validation Coarse Accuracy: 0.0488


In [13]:
## test set
import numpy as np
from common.functions import softmax
from common.coarse_mapping import (
    get_cifar100_fine_to_coarse_dict,
    CoarseMapper,
    compute_fine_class_accuracy,
    compute_coarse_accuracy
)

# 1. 매핑 딕셔너리 및 매퍼 준비
f2c = get_cifar100_fine_to_coarse_dict()
mapper = CoarseMapper(f2c)

# 2. validation set으로 fine accuracy → weight 계산
val_logits = model.predict(x_valid)              # (num_val, 100)
val_probs = softmax(val_logits)
val_preds = np.argmax(val_probs, axis=1)
fine_weights = compute_fine_class_accuracy(val_preds, t_valid_fine)  # (100,)

# 3. test set 예측
test_logits = model.predict(x_test)              # (num_test, 100)
test_probs = softmax(test_logits)
test_fine_preds = np.argmax(test_probs, axis=1)
true_coarse = np.array([f2c[f] for f in t_test_fine])  # 정답 coarse

# 4. coarse mapping 방식별 평가
# [1] Argmax Mapping
pred_coarse_1 = mapper.argmax_mapping(test_probs)
acc1 = compute_coarse_accuracy(pred_coarse_1, true_coarse)
print(f"[🎯 방식 1 - Argmax Mapping]     Test Coarse Accuracy: {acc1:.4f}")

# [2] Entropy Weighted Mapping
pred_coarse_2 = mapper.entropy_weighted_mapping(test_probs)
acc2 = compute_coarse_accuracy(pred_coarse_2, true_coarse)
print(f"[🔍 방식 2 - Entropy Weighted]   Test Coarse Accuracy: {acc2:.4f}")

# [3] Soft Average Mapping
pred_coarse_3 = mapper.soft_average_mapping(test_probs)
acc3 = compute_coarse_accuracy(pred_coarse_3, true_coarse)
print(f"[🧪 방식 3 - Soft Average]        Test Coarse Accuracy: {acc3:.4f}")

# [4] Validation Guided Mapping (with val weights)
pred_coarse_4 = mapper.validation_guided_mapping(test_probs, fine_weights)
acc4 = compute_coarse_accuracy(pred_coarse_4, true_coarse)
print(f"[📊 방식 4 - Validation Guided]  Test Coarse Accuracy: {acc4:.4f}")


[🎯 방식 1 - Argmax Mapping]     Test Coarse Accuracy: 0.0500
[🔍 방식 2 - Entropy Weighted]   Test Coarse Accuracy: 0.0500
[🧪 방식 3 - Soft Average]        Test Coarse Accuracy: 0.0500
[📊 방식 4 - Validation Guided]  Test Coarse Accuracy: 0.0500
